In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# --------------------------------------
# 1) New Bronze Delta Table Name (incremental, sicinc)
# --------------------------------------
bronze_table_inc = "sicinc.bronze.enppi_smart_data_inc"

In [0]:
bronze_path = "abfss://enppi-row-data@sicdatalakeenppi.dfs.core.windows.net/"
try:
    last_processed_timestamp = spark.table(bronze_table_inc)\
        .agg(F.max(F.col("ingestion_timestamp"))).collect()[0][0]
except Exception:
    last_processed_timestamp = None

In [0]:
if last_processed_timestamp:
    bronze_new_df = spark.read.format("parquet").load(bronze_path)\
        .filter(F.col("ingestion_timestamp") > last_processed_timestamp)
else:
    bronze_new_df = spark.read.format("parquet").load(bronze_path)

In [0]:
if not spark.catalog.tableExists(bronze_table_inc):
    bronze_new_df.write.format("delta").mode("overwrite").saveAsTable(bronze_table_inc)
else:
    # Merge incremental
    bronze_delta = DeltaTable.forName(spark, bronze_table_inc)
    bronze_delta.alias("target").merge(
        bronze_new_df.alias("source"),
        "target.sensor_id = source.sensor_id AND target.event_timestamp = source.event_timestamp"
    ).whenNotMatchedInsertAll().execute()